# In-Context Learning
* Be careful when doing this, as the cost is very high.

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git@develop

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.47


In [3]:
from dotenv import load_dotenv

# .env ファイルを読み込む
load_dotenv(override=True)

True

In [4]:
import pandas as pd
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import ICLClassifier

In [29]:
# pseudo
#seed = 3655
#seed = 3656
seed = 3657
train_data = 300

file_name = f'pseudodata_{seed}_{train_data}_train.csv'

i = 200

df = pd.read_csv(f'../data/pseudodata/{file_name}').head(i)
x_train = df.drop('target', axis=1)
y_train = df['target']
print(len(x_train))

200


In [63]:
# titanic
#seed = 3655
#seed = 3656
seed = 3657
train_data = 300
i = 50

df = pd.read_csv(f'../data/titanic/titanic_{seed}_{train_data}_train.csv').head(i)
x_train = df.drop('survived', axis=1)
y_train = df['survived']
print(len(x_train))


50


In [64]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {'columns_name': True}

iclm = ICLClassifier(llm_model = llm_model)

### Make prompt

In [65]:
icl_prompt = iclm.fit(x_train, y_train)

Number of input tokens:7301


In [66]:
print(icl_prompt)


        Predict the probability value by observing all of the following conditions.
        ・Output must be numeric only.
        ・Do not output any text.
        ・Predicting probability values as finely as possible increases overall accuracy.
        ・Please refer to the following data to estimate the probability that "target" will be 1. The rightmost column with a value of 0 or 1 is 'target'.
        ------------------
        3.0,18.0,0.0,1.0,9.35,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3.0,31.0,0.0,0.0,7.775,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2.0,3.0,1.0,1.0,26.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
2.0,26.0,1.0,1.0,26.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2.0,34.0,0.0,1.0,23.0,1.0,0.0,

### Prediction

In [33]:
file_name = f'pseudodata_{seed}_{train_data}_test.csv'


df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_test = df.drop('target', axis=1)
y_test = df['target']

In [67]:
df = pd.read_csv(f'../data/titanic/titanic_{seed}_{train_data}_test.csv')
x_test = df.drop('survived', axis=1)
y_test = df['survived']

In [68]:
y_proba = iclm.predict(x_test)

y_pred = (y_proba > 0.5).astype(int)

output = pd.DataFrame(y_proba, columns=['pred'])
#data_name = 'pseudodata'
data_name = 'titanic'
output.to_csv(f'../data/icl_output/pred_{data_name}_{seed}_{i}.csv')

100%|██████████| 300/300 [13:06<00:00,  2.62s/it]


In [70]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(accuracy, 4)}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {round(precision, 4)}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {round(recall, 4)}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {round(f1, 4)}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {round(roc_auc, 4)}')

Accuracy: 0.8
Precision: 0.697
Recall: 0.697
F1 score: 0.697
ROC-AUC: 0.8431


In [8]:
columns = 4
train_data = 300

file_name = f'pseudodata_{columns}f_{train_data}_test.csv'

df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_test = df.drop('target', axis=1)
y_test = df['target']

prefix = 1
y_proba = pd.read_csv(f'../data/icl_output/pred.csv')
y_proba = y_proba.drop('Unnamed: 0', axis=1)
y_pred = (y_proba > 0.5).astype(int)

In [9]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

ValueError: Found input variables with inconsistent numbers of samples: [700, 300]